# Multi-Armed Bandits (MAB)

## Stochastic MAB

É o problema básico de MAB. O algoritmo deve escolher entre K ações em T rodadas. Cada ação está ligada com uma distribuição de recompensa, que não muda ao longo das rodadas. O objetivo é descobrir a ação que traz a maior média de recompensas sem perder muito tempo explorando, obtendo maiores recompensas médias ao longo das T rodadas.

### Carregando bibliotecas

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
from tqdm import tqdm
from plotly.offline import init_notebook_mode
from sklearn.model_selection import ParameterGrid
init_notebook_mode(connected=True)

### Definindo constantes

In [2]:
SEED = 1234  # Semente para geração de números aleatórios
NUM_ARMS = 10  # Número de braços do ambiente MAB testados
NUM_ROUNDS = 1000  # Número de rodadas do ambiente MAB testados a cada episódio
NUM_EPISODES = 1000  # Número de episódios de simulação

np.random.seed(seed=SEED)

### Criando classes básicas

#### Ambiente

Um ambiente de MAB é um conjunto de braços (arms, ações, actions) que o agente (algoritmo) pode escolher para interagir.

Cada braço possui uma probabilidade de recompensa associada, que é desconhecida para o agente. 

Por exemplo, um braço com probabilidade de recompensa `0.5` significa que o agente tem `50%` de chance de receber uma recompensa (retornar 1) ao interagir com esse braço, e `50%` de chance de não receber uma recompensa (retornar 0). Outro exemplo, caso o braço tenha probabilidade de recompensa `0.8`, o agente tem `80%` de chance de receber uma recompensa ao interagir com esse braço e `20%` de não receber.

In [3]:
class MABEnvironment:
        
    def __init__(self, num_arms: int):
        '''
        num_arms: int - Número de braços (arms) no ambiente
        '''
        self.num_arms = num_arms
        self.reward_distributions = np.random.uniform(low=0, high=1, size=num_arms)  # Probabilidade de recompensa de cada braço, é um valor aleatório entre 0 e 1 - ex. self.reward_distributions[0] = 0.8, significa que o braço 1 tem 80% de chance de recompensa (retornar 1)

    def step(self, action: int) -> int:
        '''
        Realiza uma interação com o ambiente, escolhendo um braço para interagir e recebendo uma recompensa. A recompensa seguirá a distribuição de probabilidade associada ao braço.

        action: int - O índice do braço que o agente escolheu interagir
        '''
        return np.random.choice([0, 1], p=[1-self.reward_distributions[action], self.reward_distributions[action]])

    def get_best_arm_reward_prob(self) -> float:
        '''
        Retorna a probabilidade de recompensa do melhor braço (braço com maior probabilidade de recompensa).
        '''
        return np.max(self.reward_distributions)
    
    def get_best_arm_index(self) -> int:
        '''
        Retorna o índice do melhor braço (braço com maior probabilidade de recompensa).
        '''
        return np.argmax(self.reward_distributions)

    def display(self):
        '''
        Exibe um gráfico de barras com a distribuição de probabilidade de recompensa de cada braço. O braço com maior probabilidade de recompensa é pintado de verde.

        Também exibe uma tabela com as probabilidades de recompensa de cada braço e um resumo estatístico dessas probabilidades.
        '''
        best_arm_index = self.get_best_arm_index()

        # Dataframe com as probabilidades de recompensa de cada braço. Usado tanto para exibir o gráfico de barras quanto a tabela
        df = pd.DataFrame({
            'Arm': [str(x+1) for x in range(self.num_arms)],
            'Reward Distribution': self.reward_distributions
        })

        # Gerando o gráfico de barras interativo
        fig = px.bar(df, x='Arm', y='Reward Distribution', title='Reward Distribution of Arms')
        fig['data'][0]['marker']['color'] = ['blue' if x != best_arm_index else 'green' for x in range(self.num_arms)]  # Pinta de verde o melhor braço
        fig.show()
        
        # Exibindo a tabela com as probabilidades de recompensa de cada braço e um resumo estatístico dessas probabilidades
        display(df)
        display(df.describe())

env = MABEnvironment(num_arms=NUM_ARMS)

In [4]:
env.display()

,Arm,Reward Distribution
0,1,0.191519
1,2,0.622109
2,3,0.437728
3,4,0.785359
4,5,0.779976
5,6,0.272593
6,7,0.276464
7,8,0.801872
8,9,0.958139
9,10,0.875933


,Reward Distribution
count,10.000000
mean,0.600169
std,0.282342
min,0.191519
25%,0.316780
50%,0.701042
75%,0.797744
max,0.958139


#### Algoritmo (agente)

Classe abstrata para algoritmos de MAB. Todos os algoritmos devem herdar dessa classe e implementar os métodos abstratos.

Os algoritmos de MAB devem escolher um braço (do ambiente) para interagir em cada interação. Após a interação, uma recompensa (0 ou 1) é recebida do ambiente e o algoritmo pode atualizar suas informações internas para melhorar suas escolhas no futuro.

In [5]:
from abc import ABC, abstractmethod

class MABAlgorithm(ABC):

    @abstractmethod
    def __init__(self, num_arms: int):
        pass

    @abstractmethod
    def update(self, action: int, reward: int):
        '''
        Atualiza informações internas após receber uma recompensa por uma ação.

        action: int - O índice do braço que o agente escolheu interagir

        reward: int - A recompensa recebida após interagir com o braço escolhido
        '''
        pass

    @abstractmethod
    def select_action(self) -> int:
        '''
        Seleciona um braço para interagir.
        '''
        pass
    
    def reset(self) -> int:
        '''
        Primeiro passo de um novo episódio com o ambiente. Deve ser chamado no início de cada episódio. Isso é necessário, pois na primeira interação não temos uma recompensa para atualizar as informações internas do algoritmo, como é feito no método step.
        '''
        self.last_action = self.select_action()
        return self.last_action
    
    def step(self, reward: int) -> int:
        '''
        Um passo de uma nova interação com o ambiente. Deve ser chamado a cada interação, recebendo a recompensa da interação anterior.
        '''
        self.update(self.last_action, reward)  # Atualiza as informações internas do algoritmo com a recompensa recebida da interação anterior
        self.last_action = self.select_action()  # Seleciona a ação para a interação atual, com o estado atualizado.
        return self.last_action
    

    def init_mean_rewards(self, num_arms: int):
        '''
        Inicializa as informações internas para algoritmos que precisam manter a média de recompensas de cada braço.
        '''
        self.rewards_acum = np.zeros(num_arms)   # Vetor que guarda a recompensa acumulada de cada braço
        self.actions_count = np.zeros(num_arms)  # Vetor que guarda o número de vezes que cada braço foi escolhido
        self.rewards_mean = np.zeros(num_arms)   # Vetor que guarda a média de recompensas de cada braço

    def update_mean_rewards(self, action: int, reward: int):
        '''
        Atualiza as informações internas para algoritmos que precisam manter a média de recompensas de cada braço.
        No caso, atualiza a média de recompensas de um braço específico após receber uma recompensa por uma ação, assim como o número de vezes que esse braço foi escolhido.

        action: int - O índice do braço que o agente escolheu interagir

        reward: int - A recompensa recebida após interagir com o braço escolhido
        '''
        self.rewards_acum[action] += reward
        self.actions_count[action] += 1
        self.rewards_mean[action] = self.rewards_acum[action] / self.actions_count[action]

#### Experimento

Um experimento executa um algoritmo de MAB específico em um ambiente por um número `num_episodes` de episódios. Cada episódio possui `num_rounds` rodadas (interações).

A cada rodada, o algoritmo escolhe uma ação e recebe a recompensa da ação escolhida. Após executar `num_rounds` rodadas, o episódio é finalizado e o algoritmo é reinicializado, tendo todo seu estado interno de volta para como ele estava no começo do episódio. Após executar `num_episodes` episódios, é possível verificar a média de escolha de cada ação (assim como o desvio padrão), e a recompensa média de cada rodada do algoritmo.

Também é possível fornecer um dicionário com valores para realização de busca em grade de hiperparâmetros.

In [6]:
from typing import Type

class MABExperiment:
    
    def __init__(self, num_arms: int, num_rounds: int, num_episodes: int, Algorithm: Type[MABAlgorithm], grid_params: dict, environment: MABEnvironment):
        '''
        num_arms: int - Número de braços (arms) no ambiente

        num_rounds: int - Número de interações com o ambiente em cada episódio

        num_episodes: int - Número de episódios de simulação

        build_algorithm: Type[MABAlgorithm] - Classe do algoritmo a ser testado

        grid_params: dict - Dicionário com os parâmetros a serem testados no algoritmo. As chaves são os nomes dos parâmetros e os valores são listas com os valores a serem testados. Exemplo: {'epsilon': [0.1, 0.2, 0.3], 'alpha': [0.1, 0.2, 0.3]}

        environment: MABEnvironment - Ambiente com o qual o algoritmo interage
        '''
        self.num_arms = num_arms
        self.num_rounds = num_rounds
        self.num_episodes = num_episodes
        self.Algorithm = Algorithm
        self.grid_params = grid_params
        self.env = environment

    def run(self, plot_graphics: bool=True):
        '''
        Executa o experimento de aprendizado do agente no ambiente. No caso, são `num_episodes` episódios com `num_rounds` interações, salvando-se as recompensas recebidas e a ação escolhida em cada interação. Ao final, exibe um gráfico com a média da recompensa média acumulada em cada interação ao decorrer dos `num_episodes` episódios e um gráfico de barras com o número médio de vezes que cada braço foi escolhido, caso `plot_graphics` seja True. 
        
        Retorna uma tupla com uma lista de recompensa média acumulada em cada interação e uma lista com o número de vezes que cada braço foi escolhido.

        plot_graphics: bool - Se True, exibe os gráficos ao final do experimento
        '''
        all_mean_rewards = {}  # Lista para guardar a recompensa média acumulada em cada interação de cada episódio, para todos os parâmetros testados
        all_mean_rewards_std = {}  # Lista para guardar o desvio padrão da recompensa média acumulada em cada interação de cada episódio, para todos os parâmetros testados
        all_actions_select_count_mean = {}  # Lista para guardar o número médio de vezes que cada braço foi escolhido em cada episódio, para todos os parâmetros testados
        all_actions_select_count_std = {}  # Lista para guardar o desvio padrão do número de vezes que cada braço foi escolhido em cada episódio, para todos os parâmetros testados

        grid_params = ParameterGrid(self.grid_params)
        # Cada iteração do loop é um teste com um conjunto de parâmetros
        for i, params in enumerate(grid_params):
            print(f'Teste {i+1} de {len(grid_params)}')
            print(f'Testando com os parâmetros: {params}')
            mean_rewards_matrix = np.empty((self.num_episodes, self.num_rounds))  # Matriz para guardar a recompensa média acumulada em cada interação de cada episódio. Cada linha é um episódio e cada coluna é uma rodada.
            actions_select_count_matrix = np.zeros((self.num_episodes, self.num_arms))  # Matriz para guardar o número de vezes que cada braço foi escolhido em cada episódio. Cada linha é um episódio e cada coluna é um braço.

            # Executa os episódios
            for episode in tqdm(range(self.num_episodes)):
                # Inicialização de um novo episódio
                self.alg = self.Algorithm(self.num_arms, **params)
                action = self.alg.reset()
                rewards_acum = 0

                # Executa as rodadas do episódio
                for i in range(self.num_rounds):
                    actions_select_count_matrix[episode, action] += 1
                    reward = self.env.step(action)
                    rewards_acum += reward
                    mean_rewards_matrix[episode, i] = rewards_acum / (i + 1)
                    action = self.alg.step(reward)
            
            all_mean_rewards[str(params)] = np.mean(mean_rewards_matrix, axis=0)  # Calcula a média da recompensa média acumulada em cada interação
            all_mean_rewards_std[str(params)] = np.std(mean_rewards_matrix, axis=0)  # Calcula o desvio padrão da recompensa média acumulada em cada interação
            all_actions_select_count_mean[str(params)] = np.mean(actions_select_count_matrix, axis=0)  # Calcula o número médio de vezes que cada braço foi escolhido
            all_actions_select_count_std[str(params)] = np.std(actions_select_count_matrix, axis=0)  # Calcula o desvio padrão do número de vezes que cada braço foi escolhido

        best_params = max(all_mean_rewards, key=lambda x: all_mean_rewards[x][-1])  # Parâmetros que geraram a maior recompensa média acumulada na última interação
        best_mean_rewards = all_mean_rewards[best_params]  # Recompensa média acumulada em cada interação do melhor teste
        best_mean_rewards_std = all_mean_rewards_std[best_params]  # Desvio padrão da recompensa média acumulada em cada interação do melhor teste
        best_actions_select_count_mean = all_actions_select_count_mean[best_params]  # Número médio de vezes que cada braço foi escolhido no melhor teste
        best_actions_select_count_std = all_actions_select_count_std[best_params]  # Desvio padrão do número de vezes que cada braço foi escolhido no melhor teste
        
        if plot_graphics:
            self.__plot_graphics(all_mean_rewards, all_mean_rewards_std, best_actions_select_count_mean, best_actions_select_count_std, best_params)
        
        return best_mean_rewards, best_mean_rewards_std, best_actions_select_count_mean, best_actions_select_count_std
    
    def __plot_graphics(self, all_mean_rewards: dict, all_mean_rewards_std: np.ndarray, best_actions_select_count_mean: np.ndarray, best_actions_select_count_std: np.ndarray, best_params: dict):
        '''
        Exibe um gráfico com a recompensa média acumulada em cada interação, com a linha "Best" representando a recompensa média esperada do melhor braço. Também exibe um gráfico de barras com o número de vezes que cada braço foi escolhido no melhor conjunto de hiperparâmetros.
        '''
        alg_name = self.alg.__class__.__name__  # Nome do algoritmo é o nome da classe que ele está implementado
        best_reward_prob = self.env.get_best_arm_reward_prob()
        best_arm_index = self.env.get_best_arm_index()
        
        avg_rewards_per_round_df = pd.DataFrame(columns=['Round', 'Reward', 'Type'])

        for params, mean_rewards in all_mean_rewards.items():
            # Cria um DataFrame com a recompensa média acumulada em cada interação de um teste, para cada conjunto de hipérparâmetros testado
            df_alg = pd.DataFrame({
                'Round': [x+1 for x in range(self.num_rounds)],
                'Reward': mean_rewards,
                'Type': [params for _ in range(self.num_rounds)]
            })
            avg_rewards_per_round_df = pd.concat([avg_rewards_per_round_df, df_alg], ignore_index=True)
        
        df_best = pd.DataFrame({
            'Round': [x+1 for x in range(self.num_rounds)],
            'Reward': [best_reward_prob for _ in range(self.num_rounds)],
            'Type': ['Best' for _ in range(self.num_rounds)]
        })

        avg_rewards_per_round_df = pd.concat([avg_rewards_per_round_df, df_best], ignore_index=True)

        # Plota o gráfico de evolução de recompensa média acumulada do algoritmo
        fig = px.line(avg_rewards_per_round_df, x="Round", y="Reward", color='Type', title=f"Average Reward per Round of {alg_name}")
        fig.update_yaxes(range=[0, 1])
        fig.show()
        
        # Cria um DataFrame com o número de vezes que cada braço foi escolhido
        df_actions = pd.DataFrame({
            'Arm': [str(x+1) for x in range(self.num_arms)],
            'Mean Number of Selections': best_actions_select_count_mean
        })

        # Plota o gráfico de barras com o número de vezes que cada braço foi escolhido
        fig = px.bar(df_actions, x='Arm', y='Mean Number of Selections', error_y=best_actions_select_count_std, title=f'Mean Number of Selections of Each Arm for {best_params}')
        fig['data'][0]['marker']['color'] = ['blue' if x != best_arm_index else 'green' for x in range(self.num_arms)]  # Pinta de verde o melhor braço
        
        fig.show()


### Criando e testando os algoritmos

#### Random

O algoritmo simplesmente escolhe um braço aleatório sempre.

In [7]:
class Random(MABAlgorithm):
        
    def __init__(self, num_arms: int):
        self.num_arms = num_arms

    def update(self, action: int, reward: int):
        # Não há necessidade de guardar nenhum estado neste algoritmo, ele sempre escolhe uma ação aleatória
        return

    def select_action(self) -> int:
        # Escolhe uma ação aleatória
        return np.random.choice(range(self.num_arms))

In [8]:
random_results = MABExperiment(
    num_arms=NUM_ARMS, 
    num_rounds=NUM_ROUNDS,
    num_episodes=NUM_EPISODES,
    Algorithm=Random,
    grid_params={},
    environment=env
).run()

Teste 1 de 1
Testando com os parâmetros: {}


100%|██████████| 1000/1000 [01:00<00:00, 16.66it/s]


#### Explore-first

No início, explora cada braço `num_explore_steps_per_arm` vezes. Após isso, na fase de aprofundamento, escolhe apenas a ação que obteve o melhor resultado, ou seja, a ação que obteve maior média de recompensas.

In [9]:
class ExploreFirst(MABAlgorithm):
        
    def __init__(self, num_arms: int, num_explore_steps_per_arm: int):
        self.init_mean_rewards(num_arms)
        self.num_arms = num_arms
        self.num_explore_steps_per_arm = num_explore_steps_per_arm
        self.current_arm = 0  # Braço atual que está sendo explorado
        self.best_arm = None  # Melhor braço. Será definido no final da fase de exploração

    def update(self, action: int, reward: int):
        # Fase de aprofundamento - não há mais atualizações a serem feitas.
        if self.current_arm >= self.num_arms:
            return
        
        # Fase de exploração - explora cada braço por num_explore_steps_per_arm vezes, atualizando a média de recompensas de cada ação.

        self.update_mean_rewards(action, reward)  # Atualiza a média de recompensas do braço escolhido com a recompensa recebida (e o número de vezes que ele foi escolhido)

        if self.actions_count[self.current_arm] >= self.num_explore_steps_per_arm:
            self.current_arm += 1  # Se já explorou o número de vezes necessário para aquele braço, passa para o próximo braço
            if self.current_arm >= self.num_arms:
                self.best_arm = np.argmax(self.rewards_mean)  # Após explorar todos os braços, escolhe o melhor braço

    def select_action(self) -> int:
        if self.best_arm is not None:
            # Aprofundamento - Escolhe o melhor braço encontrado na fase de exploração
            return self.best_arm
        else:
            # Exploração - Escolhe o braço atual que está sendo explorado
            return self.current_arm

In [10]:
explore_first_results = MABExperiment(
    num_arms=NUM_ARMS, 
    num_rounds=NUM_ROUNDS,
    num_episodes=NUM_EPISODES,
    Algorithm=ExploreFirst,
    grid_params={'num_explore_steps_per_arm': [5, 10, 15, 25, 50]},
    environment=env
).run()

Teste 1 de 5
Testando com os parâmetros: {'num_explore_steps_per_arm': 5}


100%|██████████| 1000/1000 [00:29<00:00, 33.64it/s]


Teste 2 de 5
Testando com os parâmetros: {'num_explore_steps_per_arm': 10}


100%|██████████| 1000/1000 [00:25<00:00, 39.88it/s]


Teste 3 de 5
Testando com os parâmetros: {'num_explore_steps_per_arm': 15}


100%|██████████| 1000/1000 [00:25<00:00, 38.82it/s]


Teste 4 de 5
Testando com os parâmetros: {'num_explore_steps_per_arm': 25}


100%|██████████| 1000/1000 [00:25<00:00, 39.67it/s]


Teste 5 de 5
Testando com os parâmetros: {'num_explore_steps_per_arm': 50}


100%|██████████| 1000/1000 [00:25<00:00, 39.80it/s]


#### Epsilon-Greedy

O algoritmo epsilon-greedy não possui fases de exploração e aprofundamento explicitos. Nesse algoritmo, é utilizado o hiperparâmetro `epsilon`, que é um valor real entre 0 e 1, definindo o quanto o algoritmo deve aprofundar ou explorar. O algoritmo irá **explorar** com probabilidade `epsilon`, e **aprofundar** com probabilidade `1 - epsilon`.

Por exemplo, se `epsilon` é colocado como 0.2, então a probabilidade de explorar é de 20% e a probabilidade de aprofundar é 80% (`1 - 0.2 = 0.8`).

No momento de **explorar**, o algoritmo irá escolher uma ação aleatória, sendo que todas ações tem a mesma probabilidade de serem escolhidas. No momento de **aprofundar**, o algoritmo irá escolher aquela que obteve a maior média de recompensas até o momento.

In [11]:
class EpsilonGreedy(MABAlgorithm):
        
    def __init__(self, num_arms: int, epsilon: float):
        self.init_mean_rewards(num_arms)
        self.num_arms = num_arms
        self.epsilon = epsilon

    def update(self, action: int, reward: int):
        # Atualiza a média de recompensas do braço escolhido com a recompensa recebida
        self.update_mean_rewards(action, reward)

    def select_action(self) -> int:
        if np.random.uniform() < self.epsilon:
            # Exploração - escolhe um braço aleatório
            return np.random.choice(range(self.num_arms))
        else:
            # Aprofunamento - escolhe o braço com a maior média de recompensas
            return np.argmax(self.rewards_mean)

In [12]:
epsilon_greedy_results = MABExperiment(
    num_arms=NUM_ARMS, 
    num_rounds=NUM_ROUNDS,
    num_episodes=NUM_EPISODES,
    Algorithm=EpsilonGreedy,
    grid_params={'epsilon': [0.1, 0.2, 0.3, 0.4, 0.5]},
    environment=env
).run()

Teste 1 de 5
Testando com os parâmetros: {'epsilon': 0.1}


100%|██████████| 1000/1000 [00:38<00:00, 25.88it/s]


Teste 2 de 5
Testando com os parâmetros: {'epsilon': 0.2}


100%|██████████| 1000/1000 [00:40<00:00, 24.62it/s]


Teste 3 de 5
Testando com os parâmetros: {'epsilon': 0.3}


100%|██████████| 1000/1000 [00:42<00:00, 23.28it/s]


Teste 4 de 5
Testando com os parâmetros: {'epsilon': 0.4}


100%|██████████| 1000/1000 [00:44<00:00, 22.32it/s]


Teste 5 de 5
Testando com os parâmetros: {'epsilon': 0.5}


100%|██████████| 1000/1000 [00:46<00:00, 21.58it/s]


#### Decreasing Epsilon-greedy

É uma modificação do algoritmo anterior (`Epsilon-greedy`). Nesta versão, a mesma ideia de `exploração` e `aprofundamento` é mantida do algoritmo anterior, a única mudança é que o valor de `epsilon` irá decrementar a cada rodada. Com isso, no ínicio dos experimentos, a probabilidade de ocorrer uma exploração é maior do que no final do experimento.

No caso, é definido um novo hiperparâmetro `alpha`, um valor real entre 0 e 1  (`0 < alpha < 1`). A cada interação, o valor de `epsilon` é multiplicado pelo valor de `alpha`, fazendo com que `epsilon` decremente a cada rodada.

In [13]:
class DecreasingEpsilonGreedy(MABAlgorithm):
        
    def __init__(self, num_arms: int, epsilon: float, alpha: float):
        self.init_mean_rewards(num_arms)
        self.num_arms = num_arms
        self.epsilon = epsilon
        self.alpha = alpha

    def update(self, action: int, reward: int):
        # Atualiza a taxa de exploração (episilon) multiplicando-a por alpha (decaimento)
        self.epsilon *= self.alpha
        # Atualiza a média de recompensas do braço escolhido com a recompensa recebida
        self.update_mean_rewards(action, reward)

    def select_action(self) -> int:
        if np.random.uniform() < self.epsilon:
            # Exploração - escolhe um braço aleatório
            return np.random.choice(range(self.num_arms))
        else:
            # Aprofunamento - escolhe o braço com a maior média de recompensas
            return np.argmax(self.rewards_mean)

In [14]:
decreasing_epsilon_greedy_results = MABExperiment(
    num_arms=NUM_ARMS, 
    num_rounds=NUM_ROUNDS,
    num_episodes=NUM_EPISODES,
    Algorithm=DecreasingEpsilonGreedy,
    grid_params={'epsilon': [1.0, 0.75, 0.5], 'alpha': [0.995, 0.99, 0.98, 0.95]},
    environment=env
).run()

Teste 1 de 12
Testando com os parâmetros: {'alpha': 0.995, 'epsilon': 1.0}


100%|██████████| 1000/1000 [00:40<00:00, 24.85it/s]


Teste 2 de 12
Testando com os parâmetros: {'alpha': 0.995, 'epsilon': 0.75}


100%|██████████| 1000/1000 [00:40<00:00, 24.81it/s]


Teste 3 de 12
Testando com os parâmetros: {'alpha': 0.995, 'epsilon': 0.5}


...

100%|██████████| 1000/1000 [00:37<00:00, 26.32it/s]


Teste 11 de 12
Testando com os parâmetros: {'alpha': 0.95, 'epsilon': 0.75}


100%|██████████| 1000/1000 [00:38<00:00, 26.12it/s]


Teste 12 de 12
Testando com os parâmetros: {'alpha': 0.95, 'epsilon': 0.5}


100%|██████████| 1000/1000 [00:38<00:00, 25.95it/s]


####  Upper Confidence Bound (UCB)

O algoritmo UCB tenta evitar, no momento de **exploração**, testar ações que já provaram dar resultados ruins. Ou seja, ao invés de simplesmente escolher uma ação aleatória no momento de **exploração**, este algoritmo irá adotar uma estratégia de **exploração** mais "inteligente", buscando explorar ações que ainda não se sabe muito sobre.

No caso, o algoritmo é implementado utilizando a fórmula abaixo:

$$
  UCB(a) = Q(a) + c\sqrt{\frac{2\ln(t)}{N(a)}}
$$

Está equação gera uma pontuação para cada ação disponível a cada interação. A ação escolhida pelo algoritmo é aquela que obter a maior pontuação. Por exemplo, `UCB(1) = 1`, quer dizer que o braço 1 obteve pontuação igual a 1. Em um outro exemplo, se tivermos apenas 3 braços e tivermos as pontuações `UCB(1) = 0.7`, `UCB(2) = 1.0` e `UCB(3) = 0.5`, a ação 2 seria escolhida pelo algoritmo, pois obteve a maior pontuação.

Entrando em mais detalhes desta equação, o valor `Q(a)` é igual à média das recompensas obtidas pela ação específica. Exemplificando, `Q(1) = 0.7` significa que o braço 1 possui média de recompensas igual a 0.7. Essa parte da equação está relacionada ao **aprofundamento**, aumentando a probabilidade de ações com bons resultados de serem escolhidas.

O restante da equação está relacionado à **exploração**, que aumenta a pontuação de ações que ainda foram pouco testadas. O valor `c` é um hiperparâmetro, e quão maior for, maior a chance de escolher braços de **"exploração"**. O valor `t` é o número de rodadas ocorridas até o momento e o valor `N(a)` é o número de vezes que aquela ação específica já foi escolhida até o momento. 

Agora, com um exemplo, vamos verificar como essa parte da equação influencia a pontuação final.

Considerando apenas essa parte da equação:

$$
c\sqrt{\frac{2\ln(t)}{N(a)}}
$$

com apenas 3 ações possíveis de serem escolhidas, sendo `c = 0.5`, `t = 100`, `N(1) = 70`, `N(2) = 20` e `N(3) = 10`, obtemos as seguintes pontuações:

- Para a ação `1`, temos a pontuação `0.1195`
- Para a ação `2`, temos a pontuação `0.2236`
- Para a ação `3`, temos a pontuação `0.3162`

Logo, é possível observar que o segundo termo da equação favorece ações que ainda foram pouco exploradas, já que ações menos escolhidas (valor de `N` menor), obtem maiores valores neste termo.

In [15]:
class UCB(MABAlgorithm):
        
    def __init__(self, num_arms: int, c: float):
        self.init_mean_rewards(num_arms)
        self.num_arms = num_arms
        self.c = c
        self.current_test_arm = 0  # É o braço que está sendo testado no momento (até testar todos os braços uma vez)
        self.ucb_values = None  # Será um vetor com os valores de UCB para cada braço

    def update(self, action: int, reward: int):
        self.update_mean_rewards(action, reward)
        total_actions_count = np.sum(self.actions_count)

        if self.current_test_arm < self.num_arms:  
            # Precisa testar todos os braços uma vez antes de começar a usar o UCB. Caso não fosse feito esta etapa, divisões por zero poderiam ocorrer.
            self.current_test_arm += 1
        else:
            # Calcula os valores de UCB para cada braço.
            self.ucb_values = self.rewards_mean + self.c * np.sqrt(2 * np.log(total_actions_count) / self.actions_count)

    def select_action(self) -> int:
        if self.current_test_arm < self.num_arms:
            # Testa cada braço uma vez antes de começar a usar o UCB de fato.
            return self.current_test_arm
        else:
            # Escolhe o braço com o maior valor de UCB
            return np.argmax(self.ucb_values)

In [16]:
UCB_results = MABExperiment(
    num_arms=NUM_ARMS, 
    num_rounds=NUM_ROUNDS,
    num_episodes=NUM_EPISODES,
    Algorithm=UCB,
    grid_params={'c': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.5, 2.0]},
    environment=env
).run()

Teste 1 de 12
Testando com os parâmetros: {'c': 0.1}


100%|██████████| 1000/1000 [00:51<00:00, 19.55it/s]


Teste 2 de 12
Testando com os parâmetros: {'c': 0.2}


100%|██████████| 1000/1000 [00:50<00:00, 19.70it/s]


Teste 3 de 12
Testando com os parâmetros: {'c': 0.3}


...

100%|██████████| 1000/1000 [00:51<00:00, 19.47it/s]


Teste 11 de 12
Testando com os parâmetros: {'c': 1.5}


100%|██████████| 1000/1000 [00:51<00:00, 19.56it/s]


Teste 12 de 12
Testando com os parâmetros: {'c': 2.0}


100%|██████████| 1000/1000 [01:00<00:00, 16.43it/s]


#### Comparando os resultados de todos os algoritmos

In [17]:
# Desempacotando os resultados

# Random
random_mean_rewards, random_mean_rewards_std, random_actions_select_count_mean, random_actions_select_count_std = random_results

# Explore first
explore_first_mean_rewards, explore_first_mean_rewards_std, explore_first_actions_select_count_mean, explore_first_actions_select_count_std = explore_first_results

# Episilon greedy
epsilon_greedy_mean_rewards, epsilon_greedy_mean_rewards_std, epsilon_greedy_actions_select_count_mean, epsilon_greedy_actions_select_count_std = epsilon_greedy_results

# Decreasing episilon greedy
decreasing_epsilon_greedy_mean_rewards, decreasing_epsilon_greedy_mean_rewards_std, decreasing_epsilon_greedy_actions_select_count_mean, decreasing_epsilon_greedy_actions_select_count_std = decreasing_epsilon_greedy_results

# UCB
UCB_mean_rewards, UCB_mean_rewards_std, UCB_actions_select_count_mean, UCB_actions_select_count_std = UCB_results

In [18]:
# Plotando os resultados médios de recompensa por rodada
dfs = []
algs_dict = {
    'best': [env.get_best_arm_reward_prob() for _ in range(NUM_ROUNDS)],
    'random': random_mean_rewards,
    'explore_first': explore_first_mean_rewards,
    'epsilon_greedy': epsilon_greedy_mean_rewards,
    'decreasing_epsilon_greedy': decreasing_epsilon_greedy_mean_rewards,
    'UCB': UCB_mean_rewards
}

for alg_name, rewards in algs_dict.items():
    dfs.append(pd.DataFrame({
        'Round': [x+1 for x in range(NUM_ROUNDS)],
        'Reward': rewards,
        'Type': [alg_name for _ in range(NUM_ROUNDS)]
    }))

df = pd.concat(dfs, ignore_index=True)

fig = px.line(df, x="Round", y="Reward", color='Type', title="Average Reward per Round")
fig.update_yaxes(range=[0, 1])
fig.show()

In [19]:
# Plotando a recompensa média final de cada algoritmo

df_final_rewards = pd.DataFrame({
    'Algorithm': ['Random', 'Explore First', 'Epsilon Greedy', 'Decreasing Epsilon Greedy', 'UCB', 'Best'],
    'Final Reward': [random_mean_rewards[-1], explore_first_mean_rewards[-1], epsilon_greedy_mean_rewards[-1], decreasing_epsilon_greedy_mean_rewards[-1], UCB_mean_rewards[-1], env.get_best_arm_reward_prob()]
})

error_y_values = [random_mean_rewards_std[-1], explore_first_mean_rewards_std[-1], epsilon_greedy_mean_rewards_std[-1], decreasing_epsilon_greedy_mean_rewards_std[-1], UCB_mean_rewards_std[-1], 0]

fig = px.bar(df_final_rewards, x='Algorithm', y='Final Reward', error_y=error_y_values, title='Final Average Reward of Each Algorithm')
fig.show()

## Contextual MAB

### Formulação do problema

#### Contexto do usuário

O usuário será representado por um vetor de 3 posições (*u*), sendo cada posição preenchida com um valor binário (0 ou 1). Por exemplo, considerando um contexto de sistemas de recomendação de filmes, cada posição poderia ser um gênero de filme que o usuário gosta ou não. Como temos apenas 3 posições no vetor e cada uma podendo assumir dois possíveis valores, temos um conjunto total de apenas 8 possíveis contextos de usuário.

#### Ações / arms

Neste problema, existirão 15 ações distintas, que podem ser escolhidas independentemente do contexto. Cada ação tem um vetor de 3 posições (*a<sub>x</sub>*) associada à ela, sendo que cada valor é um número real entre 0 e 1. Este vetor, por exemplo, poderia informar o quanto um filme (que seria uma ação / arm no caso de recomendação de filmes) é parecido com determinado gênero, sendo 1 um filme que se enquadra perfeitamente no gênero, e 0 um filme que não se enquadra nem um pouco no genêro. Esta informação será usada para gerar a probabilidade de recompensa positiva ao escolher à ação em um determinado contexto.

OBS: essa informação não será usada para o aprendizado dos algoritmos de MAB apresentados aqui, mas poderiam ser usadas para tal em outros algoritmos.

### Cálculo de probabilidade de recompensa

Para calcular a probabilidade de recompensa positiva para uma ação (*a<sub>x</sub>*) e contexto (*u*) específico, a seguinte formula é utilizada:

$$
\frac{|u_0 - a_{x0}| + |u_1 - a_{x1}| + |u_0 - a_{x1}|}{3} * P_{max}
$$

Além dos valores dos vetores, também é usado uma constante chamada `Pmax`, que define a probabilidade máxima de recompensa positiva que uma ação pode chegar. Por exemplo, se `Pmax` for igual à 0.8, então a ação com maior probabilidade de recompensa positiva será 0.8 (80%).

Com a formula apresentada acima, ações mais semelhantes ao contexto terão uma chance de recompensa positiva maior, enquanto que se a ação for mais dessemelhante, menor será a probabilidade de recompensa positiva.